# Tutorial for Optimized Pre-Processing for Discrimination Prevention

The goal of this tutorial is to introduce the functionality and principles of the optimized preprocessing technique of AI Fairness 360 to an interested data scientist who have a background in bias detection and mitigation but is not familiar with the technique.

Note: For background introduction to bias and mitigation, please look up this file: tutorial_credit_scoring.ipynb

## Introduction

This tutorial consists of two parts:   
**An showcase of basic functionalities of the optimized preprocessing technique** and **a brief introduction of the principles of the technique**

We will use the adult dataset as an example and analyze how the technique could mitigate the discrimination in Race for the data. We will introduce the theoritical basis and main concepts of the technique as we walk through the example.

## Reference

[adult dataset from UCI](http://archive.ics.uci.edu/ml/datasets/Adult)  
[Optimized Pre-Processing for Discrimination Prevention](https://papers.nips.cc/paper/6988-optimized-pre-processing-for-discrimination-prevention.pdf)

## Main Steps

We will have the following steps for the tutorial:

Step 1: Write import statements  
Step 2: Set bias detection options, load dataset, and split between train and test  
Step 3: Compute fairness metric on original training dataset  
Step 4: Mitigate bias by transforming the original dataset  
Step 5: Compute fairness metric on transformed training and testing dataset¶  
Step 6: Compare the accuracy of prediction

### Step1: Write import statements

We import several components from the aif360 package as well as other python packages. We import a custom version of the AdultDataset with certain features binned, metrics to check for bias, and classes related to the algorithm we will use to mitigate bias.

In [48]:
import sys
sys.path.append("../")

import numpy as np

from aif360.metrics import BinaryLabelDatasetMetric

from aif360.algorithms.preprocessing.optim_preproc import OptimPreproc
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions\
            import load_preproc_data_adult
from aif360.algorithms.preprocessing.optim_preproc_helpers.distortion_functions\
            import get_distortion_adult
from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools

from IPython.display import Markdown, display
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from common_utils import compute_metrics
from aif360.metrics import ClassificationMetric

In [14]:
np.random.seed(42)

### Step 2: Set bias detection options, load dataset, and split between train and test

In Step 2 we load the initial dataset, setting the protected attribute to be race. We then splits the original dataset into training and testing datasets. Finally, we set two variables (to be used in Step 3) for the privileged (1) and unprivileged (0) values for the race attribute. These are key inputs for detecting and mitigating bias.

In [72]:
dataset_orig = load_preproc_data_adult(['race'])

dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

privileged_groups = [{'race': 1}] # White
unprivileged_groups = [{'race': 0}] # Not white

### Step 3: Compute fairness metric on original training dataset

We will use aif360 to detect bias in the dataset. Our test is to compare the percentage of favorable results for the privileged and unprivileged groups, subtracting the former percentage from the latter. A negative value indicates less favorable outcomes for the unprivileged groups. This is implemented in the method called mean_difference on the BinaryLabelDatasetMetric class. The code below performs this check and displays the output:

In [73]:
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.105374


### Step 4: Mitigate bias by transforming the original dataset

The previous step showed that the privileged group was getting 9.8% more positive outcomes in the training dataset. Since this is not desirable, we are going to try to mitigate this bias in the training dataset by implementing optimized preprocessing algorithms. This algorithm will transform the dataset to have more equity in positive outcomes on the protected attribute for the privileged and unprivileged groups.

The algorithm requires some tuning parameters, which are set in the optim_options variable and passed as an argument along with some other parameters, including the 2 variables containg the unprivileged and privileged groups defined in Step 3.

We then call the fit and transform methods to perform the transformation, producing a newly transformed training dataset (dataset_transf_train). Finally, we ensure alignment of features between the transformed and the original dataset to enable comparisons.

**We will talk about the theoritical basis for this algorithm later.**

In [74]:
optim_options = {
    "distortion_fun": get_distortion_adult,
    "epsilon": 0.05,
    "clist": [0.99, 1.99, 2.99],
    "dlist": [.1, 0.05, 0]
}
    
OP = OptimPreproc(OptTools, optim_options)

OP = OP.fit(dataset_orig_train)
dataset_transf_train = OP.transform(dataset_orig_train, transform_Y=True)

dataset_transf_train = dataset_orig_train.align_datasets(dataset_transf_train)

Optimized Preprocessing: Objective converged to 0.000000


### Basic Ideas for the optimized preprocessing algorithm

The main idea of the optimized preprocessing algorithm is to work on a constrained optimization problem as below.

![](images/optim_preproc.png)

The optimization problem has three main focuses:

**I. Utility Preservation**

As is represented by the objective function, we require that the distribution of $(\hat X, \hat Y)$ be statistically close to the original distribution of $(X, Y)$, where D is the protected variables, X is the input variables(other than D) and Y is the output and $\hat X, \hat Y$ implicate the corresponding transformed data. This is to ensure that a
model learned from the transformed dataset (when averaged over the protected variables D) is not
too different from one learned from the original dataset. For a given dissimilarity measure $\Delta$ between probability distributions (e.g. KL-divergence), we require that the dissimilarity be small.

**II. Discrimination Control**

As is represented by the first constraint, $J(·, ·)$ denotes some distance function. Here we ask for a similar conditional probability of the target variable Y given the protected variable D as the original one.

**III. Distortion Control**

The mapping should satisfy distortion constraints with respect to the domain $X \times Y$. We assume that $\delta(x, y, x, y) = 0$ for all $(x, y) \in X \times Y$, else 1. These constraints restrict the mapping to reduce or avoid altogether certain large changes.

### Step 5: Compute fairness metric on transformed training and testing dataset¶

Now that we have a transformed dataset, we can check how effective it was in removing bias by using the same metric we used for the original training and testing dataset in Step 3. Once again, we use the function mean_difference in the BinaryLabelDatasetMetric class:

In [75]:
metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                                               unprivileged_groups=unprivileged_groups,
                                               privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())

#### Transformed training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.049406


In [76]:
# Load, clean up original test data and compute metric
dataset_orig_test = dataset_transf_train.align_datasets(dataset_orig_test)
display(Markdown("#### Testing Dataset shape"))
print(dataset_orig_test.features.shape)

metric_orig_test = BinaryLabelDatasetMetric(dataset_orig_test, 
                                         unprivileged_groups=unprivileged_groups,
                                         privileged_groups=privileged_groups)
display(Markdown("#### Original test dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_test.mean_difference())
#Transform test data and compute metric
dataset_transf_test = OP.transform(dataset_orig_test, transform_Y = True)
dataset_transf_test = dataset_orig_test.align_datasets(dataset_transf_test)

metric_transf_test = BinaryLabelDatasetMetric(dataset_transf_test, 
                                         unprivileged_groups=unprivileged_groups,
                                         privileged_groups=privileged_groups)
display(Markdown("#### Transformed test dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_test.mean_difference())


#### Testing Dataset shape

(14653, 18)


#### Original test dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.092213


#### Transformed test dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.038322


### Step6: Compare accuracy of the prediction

We would like to see if the preprocssing of data would hurt the performance of the model. So we trained a simple logistic model on both original and transformed training data, then compare the prediction accuracy on testing data.

In [83]:
# Logistic regression classifier and predictions
scale_orig = StandardScaler()
X_train = scale_orig.fit_transform(dataset_orig_train.features)
y_train = dataset_orig_train.labels.ravel()

lmod = LogisticRegression(solver='lbfgs')
lmod.fit(X_train, y_train)

X_test = scale_orig.transform(dataset_orig_test.features)
y_test = dataset_orig_test.labels.ravel()
y_test_pred = lmod.predict(X_test)
acc_orig = np.mean(y_test_pred == y_test)

print("Accuracy on original test data by logistic regression is = %f" % acc_orig)

Accuracy on test data by logistic regression is = 0.799017


In [84]:
# Logistic regression classifier and predictions
scale_transf = StandardScaler()
X_train = scale_transf.fit_transform(dataset_transf_train.features)
y_train = dataset_transf_train.labels.ravel()

lmod = LogisticRegression(solver='lbfgs')
lmod.fit(X_train, y_train)

X_test = scale_transf.transform(dataset_transf_test.features)
y_test = dataset_transf_test.labels.ravel()
y_test_pred = lmod.predict(X_test)
acc_transf = np.mean(y_test_pred == y_test)

print("Accuracy on transformed test data by logistic regression is = %f" % acc_transf)

Accuracy on transformed test data by logistic regression is = 0.797652


We may observe that the accuracy has decreased by approximately 0.2\%, which is acceptable. We have applied similar workflow on different datasets and use AUC to evaluate the model performance.

The figure below presents the relationship between discrimination and model performance.
![](images/optim_preproc2.png)

## Conclusions

The optimized preprocessing provides an algorithm to mitigate potential bias of the training data and as well maintain the effectiveness of data. The algorithm is straightforward by seeing its optimization function, which consists of an objective function with two constrains, which corresponds to the three focuses of the algorithm: **Utility Preservation, Discrimination Control, and Distortion Control**. 

The algorithm works good to reduce bias on the protected variables as a preprocessing technique. However, we may still have to sacrifice some model performance as we reduce discrimination. We would suggest that users should carefully choose a proper parameter to control the level of reduce of bias. For example, in some field where human is involved along with ethical issues, we may rather sacrifice model efficiency to get an unbiased model, while in other cases we may focus on the performance of the model instead.

Also, optimized preprocessing is just one technique that we may use to mitigate bias. Other techniques for inprocessing and postprocessing should also be considered while building the model.